In [15]:
import torch
import skorch
from tqdm.notebook import tqdm
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, WeightedRandomSampler

#### PYTORCH CONFIGURATION SETTINGS ######
device = "cpu"#"mps" if torch.has_mps else "cpu"
print(f"Using {device} device")
torch.manual_seed(56)

Using cpu device


### Logistic Regression PyTorch implementation

In [16]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
def train_loop(dataloader, model, loss_fn, optimizer, print_nth_batch=4):
    size = len(dataloader.dataset)
    print("yay training")
    for batch, (X, y) in enumerate(dataloader):
        # print(y)
        # Compute prediction and loss
        pred = torch.squeeze(model(X))
        # print(pred)
        # regularization, computing largest singular value
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % print_nth_batch == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    # we don't want to track gradients here because we're just doing
    # a forward pass to evaluate predictions
    with torch.no_grad():
        for X, y in dataloader:
            pred = torch.squeeze(model(X))
            test_loss += loss_fn(pred, y).item()
            # round predicted probs to get label prediction, compute n correct
            correct += (pred.round() == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
def test_auc_score(dataset, model):
    X, y = dataset[:]
    pred = model(X)
    pred, y = pred.detach().numpy(), y.detach().numpy()
    score = roc_auc_score(y_true=y, y_score=pred)
    print("AUC:", score)
    return score
    
    
    
class LogisticRegressionTorch(torch.nn.Module):
    def __init__(self, input_dim, output_dim=1):
        super().__init__()
        self.linear = torch.nn.Sequential(
            torch.nn.Linear(input_dim, output_dim, bias=True),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        # logits = torch.sigmoid(self.linear(x))
        probs = self.linear(x)
        return probs

In [91]:
X_1 = torch.rand(1000, 128) + .2
y_1 = torch.zeros(1000,)
X_2 = torch.rand(1000, 128) - .1
y_2 = torch.ones(1000,)

X = torch.cat((X_1, X_2))
y = torch.cat((y_1, y_2))

dataset = SimpleDataset(X, y)

train_prop = .8
train_num = int(train_prop * len(dataset))
test_num = len(X) - train_num


train_set, test_set = torch.utils.data.random_split(dataset,
                                                    [train_num, test_num])


target = train_set.dataset.y[train_set.indices]
cls_weights = torch.from_numpy(
    compute_class_weight(class_weight='balanced',
                         classes=np.unique(target.numpy()),
                         y=target.numpy())
)
weights = cls_weights[target.numpy()]
sampler = WeightedRandomSampler(weights, len(target.numpy()), replacement=True)

train_dataloader = DataLoader(train_set, batch_size=100, sampler=sampler)
test_dataloader = DataLoader(test_set, batch_size=100)

In [585]:
model = LogisticRegressionTorch(128)

loss_fn = torch.nn.BCELoss()
weight_decay = 0
lr = 5e-1
optimizer = torch.optim.SGD(model.parameters(), lr=lr,weight_decay=weight_decay)

In [13]:
%%time
epochs = 15
for t in range(epochs):
    print(f"{'-'*30}\nEpoch {t+1}\n{'-'*30}")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(train_dataloader, model, loss_fn)
print("Done!")

------------------------------
Epoch 1
------------------------------
yay training
loss: 11.448199  [    0/12672]
loss: 19.501102  [  400/12672]
loss: 11.889750  [  800/12672]
loss: 10.050867  [ 1200/12672]
loss: 4.415318  [ 1600/12672]
loss: 20.606892  [ 2000/12672]
loss: 23.693605  [ 2400/12672]
loss: 6.622545  [ 2800/12672]
loss: 22.063299  [ 3200/12672]
loss: 9.026217  [ 3600/12672]
loss: 12.450096  [ 4000/12672]
loss: 13.314853  [ 4400/12672]
loss: 10.686407  [ 4800/12672]
loss: 17.673269  [ 5200/12672]
loss: 13.940762  [ 5600/12672]
loss: 16.124735  [ 6000/12672]
loss: 14.201858  [ 6400/12672]
loss: 16.571611  [ 6800/12672]
loss: 10.381608  [ 7200/12672]
loss: 7.627161  [ 7600/12672]
loss: 9.125181  [ 8000/12672]
loss: 8.847769  [ 8400/12672]
loss: 13.405208  [ 8800/12672]
loss: 19.638165  [ 9200/12672]
loss: 14.617030  [ 9600/12672]
loss: 7.883918  [10000/12672]
loss: 15.535515  [10400/12672]
loss: 13.479076  [10800/12672]
loss: 19.351795  [11200/12672]
loss: 16.983923  [11600/1

In [587]:
[p for p in model.parameters()][0].detach().numpy()

array([[-0.2556009 , -0.11014569, -0.34770104, -0.05363564, -0.32045805,
        -0.20047373,  0.04354731, -0.07021297, -0.2549383 , -0.08638977,
        -0.12272437, -0.26145813, -0.132995  , -0.5590581 , -0.13800876,
        -0.39361808, -0.18007791, -0.14691398, -0.11113048,  0.02109414,
        -0.02971033, -0.06245779, -0.23700692, -0.0312883 , -0.05845731,
         0.14545608, -0.22800045,  0.13870333, -0.2055197 , -0.5122873 ,
        -0.02531125,  0.07469609,  0.00397547, -0.01759478, -0.16063288,
        -0.04735122, -0.12375901, -0.19227822, -0.05999327, -0.04843619,
        -0.2967028 ,  0.00551004,  0.03163636, -0.06197273,  0.01000183,
        -0.21084203, -0.10228419, -0.07310296,  0.11168385, -0.23657438,
        -0.2252271 , -0.30507943, -0.02275092, -0.31863666, -0.14079633,
        -0.06563782,  0.15927301, -0.18031234,  0.01695587, -0.09139688,
        -0.18292908, -0.11702715, -0.5671819 ,  0.15823328, -0.11645681,
         0.01410398, -0.08147889, -0.09058848, -0.2

## Compare to sklearn

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
%%time
sk_model = LogisticRegression(penalty='l2', C=1., class_weight='balanced', fit_intercept=True)

train_idx = train_set.indices
test_idx = test_set.indices
sk_train_X = train_set.dataset.X[train_idx].detach().numpy()
sk_train_y = train_set.dataset.y[train_idx].detach().numpy()
sk_test_X = test_set.dataset.X[test_idx].detach().numpy()
sk_test_y = test_set.dataset.y[test_idx].detach().numpy()
fit = sk_model.fit(sk_train_X, sk_train_y)

sk_pred = fit.predict(sk_test_X)

AttributeError: 'SimpleDataset' object has no attribute 'indices'

In [553]:
from sklearn.metrics import accuracy_score

In [554]:
accuracy_score(sk_test_y, sk_pred)

1.0

In [ ]:
sk_model.coef_

array([[-0.37388629, -0.32047521, -0.4095046 , -0.32130418, -0.39439804,
        -0.35823253, -0.33484078, -0.33933441, -0.37543385, -0.36355242,
        -0.34292282, -0.37185879, -0.3706311 , -0.44209721, -0.36840507,
        -0.34789623, -0.41266399, -0.36281521, -0.36685014, -0.3846179 ,
        -0.31708437, -0.31090994, -0.37258387, -0.37406865, -0.36461087,
        -0.28830832, -0.41198973, -0.3104652 , -0.35061659, -0.40379479,
        -0.33055622, -0.33311162, -0.35796505, -0.35008685, -0.35916163,
        -0.35937264, -0.39998844, -0.3649274 , -0.37894635, -0.35347019,
        -0.37401855, -0.3663856 , -0.30936565, -0.37154138, -0.32723201,
        -0.36783979, -0.39499198, -0.3303809 , -0.33748294, -0.36837342,
        -0.39150569, -0.36852701, -0.3572353 , -0.37949994, -0.33189788,
        -0.40627196, -0.35212517, -0.37479089, -0.33094589, -0.37000063,
        -0.3751616 , -0.33788822, -0.37261937, -0.31014997, -0.34313105,
        -0.35801088, -0.36816738, -0.34082888, -0.3

## Using lab data

In [17]:
from ptsa.data.timeseries import TimeSeries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, LeaveOneGroupOut
from sklearn.metrics import roc_auc_score
import pandas as pd

In [18]:
ts = TimeSeries.from_hdf(
    "/Users/jrudoler/rhino_mount/scratch/jrudoler/scalp_features/LTP093_feats.h5",
)

In [19]:
X_npy = ts.data
y_npy = ts.recalled.data
X = torch.tensor(ts.data).float()
y = torch.tensor(ts.recalled.data).float()
dataset = SimpleDataset(X, y)
sessions = ts.session.values

In [22]:
train_set.y.detach().numpy().astype(int)

array([1, 1, 1, ..., 1, 1, 1])

In [21]:
logo = LeaveOneGroupOut()
for i, (train_idx, test_idx) in enumerate(logo.split(X, y, groups=sessions)):
    print(f"{'#'*30}\nSESSION {i}\n{'#'*30}")
    ## create model ##
    model = LogisticRegressionTorch(X.shape[-1])
    loss_fn = torch.nn.BCELoss()
    weight_decay = 1e-4
    lr = 5e-1
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    ## data ##
    train_set = SimpleDataset(X[train_idx], y[train_idx])
    test_set = SimpleDataset(X[test_idx], y[test_idx])
    ## class balancing ##
    cls_weights = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(train_set.y.detach().numpy()),
        y=train_set.y.detach().numpy(),
    )
    weights = cls_weights[train_set.y.detach().numpy().astype(int)]
    sampler = WeightedRandomSampler(
        weights, len(train_set.y.detach().numpy()), replacement=True
    )

    train_dataloader = DataLoader(train_set, batch_size=200, sampler=sampler)

    ## training epochs ##
    epochs = 10
    for t in range(epochs):
        print(f"{'-'*30}\nEpoch {t+1}\n{'-'*30}")
        train_loop(train_dataloader, model, loss_fn, optimizer, print_nth_batch=8)
        out = test_auc_score(train_set, model)

    lr_sk = LogisticRegression(
        C=weight_decay, class_weight="balanced", fit_intercept=True
    )
    lr_sk.fit(X=X_npy[train_idx], y=y_npy[train_idx])
    pred = lr_sk.predict_proba(X_npy[test_idx])[:, 1]
    score = roc_auc_score(y_npy[test_idx], pred)
    print("sklearn AUC:", score)

    print("Done!")


##############################
SESSION 0
##############################
------------------------------
Epoch 1
------------------------------
yay training
loss: 0.715226  [    0/12672]
loss: 28.113054  [ 1600/12672]
loss: 23.311895  [ 3200/12672]
loss: 7.153251  [ 4800/12672]
loss: 7.607974  [ 6400/12672]
loss: 11.595755  [ 8000/12672]
loss: 11.874519  [ 9600/12672]
loss: 8.548401  [11200/12672]
AUC: 0.5834685577193572
------------------------------
Epoch 2
------------------------------
yay training
loss: 14.464243  [    0/12672]
loss: 16.580606  [ 1600/12672]
loss: 5.655796  [ 3200/12672]
loss: 10.774112  [ 4800/12672]
loss: 25.277502  [ 6400/12672]
loss: 12.422085  [ 8000/12672]
loss: 30.589161  [ 9600/12672]
loss: 18.302435  [11200/12672]
AUC: 0.505155262818357
------------------------------
Epoch 3
------------------------------
yay training
loss: 18.837206  [    0/12672]
loss: 13.396523  [ 1600/12672]
loss: 26.100977  [ 3200/12672]
loss: 18.889591  [ 4800/12672]
loss: 15.965723  